In [1]:
import os
import json
import sys
from PyQt5.QtWidgets import QApplication
from src.chunk_manager_gui import ChunkManagerGUI
from src.ai import elevenlabs
from src.utils import saveB64Audio, chunkTextForTTS, create_project_folder
from src.enums import ElevenLabsTTSModel

# if not specified, will use default model from config/config.yaml
tts_model = ElevenLabsTTSModel.Turbo_v25.value

script_text = """"""
script_location = "input-test.txt"

# maybe make a copy of this notebook for F5 TTS, then combine the 2 later

def save_chunks(chunks, project_folder):
	chunks_name = os.path.join(project_folder, 'tts-chunks.json')
	with open(chunks_name, 'w') as f:
		f.write(json.dumps(chunks))


def create_audio(script,
				 audio_name='audio.mp3',
				 project_folder=None,
				 model=None):
	# TODO use request stitching to improve gaps
	# see https://elevenlabs.io/docs/api-reference/how-to-use-request-stitching
	if project_folder is None:
		project_folder = create_project_folder()

	audio = elevenlabs.getSpeechB64(script, model)
	audio_path = os.path.join(project_folder, audio_name)
	saveB64Audio(audio, audio_path)

	print(f"Audio {audio_name} created successfully in {project_folder}")
	return project_folder, audio_path


input_exists = os.path.exists(script_location)
if not script_text and not input_exists:
	raise SystemExit("Please provide a script or a script location")

if not script_text and script_location:
	with open(script_location, 'r') as f:
		script_text = f.read()

# make the text chunks
chunks = chunkTextForTTS(script_text)
project_folder = create_project_folder(reuse=True)

# save the chunks
chunks_name = os.path.join(project_folder, 'tts-chunks.json')
if not os.path.exists(chunks_name):
	print(f"Saving text chunks to {chunks_name}")
	save_chunks(chunks, project_folder)
else:
	# compare with existing chunks
	with open(chunks_name, 'r') as f:
		existing_chunks = eval(f.read())
		print('Comparing with existing text chunks')
		if existing_chunks != chunks:
			print('Chunks have changed, updating')
			save_chunks(chunks, project_folder)

# generate the tts audio files
skipped = 0
for i, chunk in enumerate(chunks):
	chunk_type = chunk['type']
	audio_name = f'{i}_{chunk_type}.mp3'
	chunk['audio'] = audio_name
	audio_exists = os.path.exists(os.path.join(project_folder, audio_name))

	if audio_exists:
		skipped += 1
		continue

	content = chunk['content']
	create_audio(content, audio_name, project_folder, tts_model)

save_chunks(chunks, project_folder)

if skipped > 0:
	print(f"Skipped generating {skipped} audio files")

# open ui to review chunks
app = QApplication(sys.argv)
ex = ChunkManagerGUI(chunks, project_folder)
ex.show()
app.exec_()
%gui qt

Audio 0_text.mp3 created successfully in project_20241018_1_0
Audio 1_title.mp3 created successfully in project_20241018_1_0
Audio 2_text.mp3 created successfully in project_20241018_1_0


2024-10-18 14:43:57.101 python[88471:2045801] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-18 14:43:57.101 python[88471:2045801] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [2]:
# elevenlabs.getModels()

In [1]:
# Standalone GUI:
# use this cell if chunks are already saved and generated

import os
import sys
from PyQt5.QtWidgets import QApplication
from src.chunk_manager_gui import ChunkManagerGUI
from src.utils import create_project_folder

project_folder = create_project_folder(reuse=True)
chunks_name = os.path.join(project_folder, 'tts-chunks.json')
with open(chunks_name, 'r') as f:
	chunks = eval(f.read())

app = QApplication(sys.argv)
ex = ChunkManagerGUI(chunks, project_folder)
ex.show()
app.exec_()
%gui qt

2024-10-18 17:57:31.445 python[93437:2159475] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-18 17:57:31.445 python[93437:2159475] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
